## SQL Table Creation

#### 1. Connect to PostgreSQL:
 Let's start by creating the connection to your PostgreSQL instance using SQLAlchemy and psycopg2.

In [49]:
# Import the DBConnection class from scripts.db_connection
import os
os.chdir(r'c:\users\ermias.tadesse\10x\Centralize-Ethiopian-medical-business-data')  # Set the working directory to the project root
from scripts.db_connection import DBConnection
from sqlalchemy import MetaData, Table, Column, Integer, String, DateTime, ForeignKey
import logging

# Create a connection instance
db = DBConnection(dbname='Central_Medical_Warehouse', user='postgres', password='Ermi@123')

# Connect to the database
db.connect()

os.chdir(r'c:\Users\ermias.tadesse\10x\Centralize-Ethiopian-medical-business-data\logs')

# Set up logging
logging.basicConfig(filename='Load_Data_To_Database.log', level=logging.INFO,
                    format='%(asctime)s %(levelname)s:%(message)s')

logging.info(f'Successfully connected to The Postgrass database: {db.dbname}')

# Initialize metadata for creating tables
metadata = MetaData()

# Define your tables
channels_table = Table('channels', metadata,
    Column('channel_id', Integer, primary_key=True, autoincrement=True),
    Column('channel_name', String, unique=True, nullable=False)
)

senders_table = Table('senders', metadata,
    Column('sender_id', Integer, primary_key=True),
    Column('sender_name', String)  # Optional if you want to track sender names
)

messages_table = Table('messages', metadata,
    Column('message_id', Integer, primary_key=True),
    Column('message', String, nullable=False),
    Column('date', DateTime, nullable=False),
    Column('sender_id', Integer, ForeignKey('senders.sender_id'), nullable=False),
    Column('channel_id', Integer, ForeignKey('channels.channel_id'), nullable=False)
)

media_table = Table('media', metadata,
    Column('media_id', Integer, primary_key=True, autoincrement=True),
    Column('media_path', String, nullable=False),
    Column('message_id', Integer, ForeignKey('messages.message_id'), nullable=False)
)

# Create the tables in the PostgreSQL database
if db.engine:
    metadata.create_all(db.engine)
    print("Tables created successfully.")
    logging.info(f'Tables created successfully.')
else:
    print("Failed to create tables due to no connection.")
    logging.info(f'Failed to create tables due to no connection.')

# # Close the connection after operations are done
# db.close()


Successfully connected to the database!
Tables created successfully.


### Database Design 
Tables:
Channels Table:

channel_id: Primary Key, auto-incremented.
channel_name: Unique name of the Telegram channel.
Senders Table:

sender_id: Primary Key.
sender_name: (optional) - if the sender has an associated name or username.
Messages Table:

message_id: Primary Key.
message: The text of the message.
date: The date the message was posted.
sender_id: Foreign Key from the Senders table.
channel_id: Foreign Key from the Channels table.
Media Table:

media_id: Primary Key, auto-incremented.
media_path: Path to the media file.
message_id: Foreign Key from the Messages table.

Explanation of Tables:
channels: Stores unique channel names.
senders: Stores unique senders based on sender_id.
messages: Stores each message, linking it to a channel and sender.
media: Stores the media paths related to messages.

## Insert Data from scraped_data_cleaned.xlsx
Next, we’ll load the cleaned Excel data, split it into the respective tables, and insert it into the PostgreSQL database.

Load Excel Data and Prepare for Insertion

In [50]:
import pandas as pd
import logging
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine
import os

# Set up logging configuration
logging.basicConfig(level=logging.INFO)

# Change to your working directory
os.chdir(r'c:\Users\ermias.tadesse\10x\Centralize-Ethiopian-medical-business-data')

# Load cleaned data from Excel
scraped_data = pd.read_excel('data/cleaned/scraped_data_cleaned.xlsx')
logging.info('Read the cleaned data from Excel')
print("Read the cleaned data from Excel.")
# Create a session using the correct engine from DBConnection
# Assuming 'db.engine' is defined somewhere in your code
Session = sessionmaker(bind=db.engine)
session = Session()
logging.info('Created a session using the correct engine from DBConnection')

# Insert channels (bulk insert)
channels = [{'channel_name': channel} for channel in scraped_data['channel_name'].unique()]
session.execute(channels_table.insert().values(channels))
# Commit the session
session.commit()
logging.info('Bulk inserted channels')
logging.info("Data inserted channels successfully.")
print("channels Data inserted successfully.")
# Insert senders (bulk insert, cast to int)
senders = [{'sender_id': int(sender_id)} for sender_id in scraped_data['sender_id'].unique()]
session.execute(senders_table.insert().values(senders))
# Commit the session
session.commit()
logging.info('Bulk inserted senders')
logging.info("Data inserted senders successfully.")
print("senders Data inserted successfully.")
# Prepare messages and media for bulk insert
messages = []
media_entries = []

for index, row in scraped_data.iterrows():
    # Get channel_id for this message
    channel_id = session.execute(
        channels_table.select().where(channels_table.c.channel_name == row['channel_name'])
    ).fetchone()[0]
    
    # Prepare message entry
    messages.append({
        'message_id': int(row['message_id']),  # Cast to int
        'message': row['message'],
        'date': row['date'],
        'sender_id': int(row['sender_id']),  # Cast to int
        'channel_id': channel_id
    })

    # Prepare media entry if media path exists
    if pd.notnull(row['media_path']):
        media_entries.append({
            'media_path': row['media_path'],
            'message_id': int(row['message_id'])  # Cast to int
        })

# Proceed with insert if all message_ids are valid
if all(-2147483648 <= message['message_id'] <= 2147483647 for message in messages):
    session.execute(messages_table.insert().values(messages))
    logging.info('Bulk inserted messages')
else:
    logging.error('Bulk insert aborted due to invalid message_id values.')

# Bulk insert media if any entries exist
if media_entries:
    session.execute(media_table.insert().values(media_entries))
    logging.info('Bulk inserted media')

# Commit the session
session.commit()
logging.info("Data inserted successfully.")

# Close the session and connection
session.close()
logging.info("Session closed.")

print("Data inserted successfully.")



Read the cleaned data from Excel.
channels Data inserted successfully.
senders Data inserted successfully.


DataError: (psycopg2.errors.NumericValueOutOfRange) integer out of range

[SQL: INSERT INTO messages (message_id, message, date, sender_id, channel_id) VALUES (%(message_id_m0)s, %(message_m0)s, %(date_m0)s, %(sender_id_m0)s, %(channel_id_m0)s), (%(message_id_m1)s, %(message_m1)s, %(date_m1)s, %(sender_id_m1)s, %(channel_id_m1)s), (%(message_id_m2)s, %(message_m2)s, %(date_m2)s, %(sender_id_m2)s, %(channel_id_m2)s), (%(message_id_m3)s, %(message_m3)s, %(date_m3)s, %(sender_id_m3)s, %(channel_id_m3)s), (%(message_id_m4)s, %(message_m4)s, %(date_m4)s, %(sender_id_m4)s, %(channel_id_m4)s), (%(message_id_m5)s, %(message_m5)s, %(date_m5)s, %(sender_id_m5)s, %(channel_id_m5)s), (%(message_id_m6)s, %(message_m6)s, %(date_m6)s, %(sender_id_m6)s, %(channel_id_m6)s), (%(message_id_m7)s, %(message_m7)s, %(date_m7)s, %(sender_id_m7)s, %(channel_id_m7)s), (%(message_id_m8)s, %(message_m8)s, %(date_m8)s, %(sender_id_m8)s, %(channel_id_m8)s), (%(message_id_m9)s, %(message_m9)s, %(date_m9)s, %(sender_

### Explanation:
Session Creation: The session is created using the engine from db.engine to manage transactions.


Data Insertion: Data from scraped_data_cleaned.xlsx is inserted into the normalized tables. The session.execute() method is used for the insertion of records into channels, senders, messages, and media tables.


Committing the Transaction: After all inserts, the session is committed to save the changes to the database.


Session and Connection Closing: Both the session and database connection are closed to ensure proper resource management.